- textloader
- date: 2020-08-06
- maintainer: YZK

In [30]:
# %%bash
# jupyter nbconvert --to script textloader.ipynb

In [4]:
%load_ext autoreload
%autoreload 2

In [4]:
import calendar
import codecs
import gc
import logging
import math
import os
import re
import sys
import multiprocessing as mp
from datetime import datetime


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import linear_model 

try:
    thisd = os.path.dirname(os.path.realpath(__file__))
    sys.path.append(thisd)
    import widgets as wg
    from parallel import runFunctionsInParallel
except Exception as E:
    if __name__ == "__main__":
        import msetup
        import widgets as wg
        from parallel import runFunctionsInParallel        
        logd = "log"
        if not os.path.exists(logd):
            os.makedirs(logd)
        msetup.setLogging(logf="{}/textloader.log".format(logd), loglv=logging.DEBUG)
    else:
        import lib.widgets as wg
        from lib.parallel import runFunctionsInParallel

    logging.warning(E)

# print(os.environ['SHELL'])

# build smarker for gridding map (combine all monthly data into one file)

In [179]:
def idSelect(stnf, fname=None, key=None, val=None, outf=None):
    '''
        all2onef: format is the same as smarker
    '''
    
    sinfo = textloader.getGI(stnf, index_col=None)
    if fname is not None:
        all2onef = pd.read_csv(fname)
        all2onef.columns = ["key"] + all2onef.columns[1:].tolist()
        all2onef.set_index("key", inplace=True)
        target = all2onef.loc[key]

        ids = sinfo["stnid"]
        sinfo.set_index(ids, inplace=True)
        sinfo = sinfo.loc[all2onef.columns[target >= val]]

        if outf is not None:
            sinfo.to_csv(outf, index=False)
    else:
        sinfo = sinfo.loc[sinfo["id"] == 1]
    
        if outf is not None:
            with open(outf, "w") as fid:
                fid.write("{0:4s} {1:6s} {2:8s} {3:7s} {4:9s} {5:10s} {6:10s} {7:2s} {8}\n".format("#SN", "StnId", "Lon", "Lat", "Elev", "t1", "t2", "Id", "StnName"))
                for idx in range(sinfo.shape[0]):
                    rc = sinfo.iloc[idx].tolist()
                    fid.write("{0:04d} {1:6s} {2:8.4f} {3:7.4f} {4:9.4f} {5:10d} {6:10d} {7:2d} {8}\n".format(rc[0], rc[1], rc[2], rc[3], rc[4], rc[5], rc[6], rc[7], rc[8]))        
    
    return sinfo

In [156]:
def mnf2onef(ind, outf, stnf, tperiod, vname, subd=None, fprefix="MONTHLY_", lb=0, stnkeep=None):
    # C1I440 新高口，同時是目標站也是某站之鄰近站，補遺的時候可能沒抓到值（因為id=1或id=2）
    
    if subd is None:
        subd = ""
    
    stninfo = textloader.getGI(stnf)
    stnid = stninfo["stnid"].values
    
    stninfo.set_index("stnid", inplace=True)
    
    nstn = len(stnid)
    syear = tperiod[0]
    eyear = tperiod[1]
    nyear = eyear - syear + 1
    Y = [syear + i for i in range(nyear)]
    m = [1 + i for i in range(12)]
    
    monthly = np.ndarray((nstn, nyear, 12))
    monthly.fill(-999)
    
    dtimes = []
    for Y_ in Y:
        for m_ in m:
            dtimes.append("{0:04d}{1:02d}".format(Y_, m_))
    
    logs1 = "mnf2onef: stnid = {}, obs = {} ({}) < {}, cxnid = {}, t1 = {}, t2 = {}"
    for iidx, id_ in enumerate(stnid):
        for yidx, Y_ in enumerate(Y):

            if len(subd) == 0:
                inf = "{0}/{1}/{2}/{3}{1}_{2}_{4}.txt".format(ind, vname, Y_, fprefix, id_)
            else:
                inf = "{0}/{1}/{2}/{3}/{4}{2}_{3}_{5}.txt".format(ind, subd, vname, Y_, fprefix, id_)

            if os.path.exists(inf):
                arr = np.loadtxt(inf, dtype={"names": ("YYYYmm", "obs"), "formats": ("i4", "f8")})
                for midx, m_ in enumerate(m):
                    Ym = Y_ * 100 + m_
                    if arr[midx][0] == Ym:
                        if arr[midx][1] < lb and stninfo.loc[stnid[iidx]]["id"] == 1:
                            logging.warning(logs1.format(stnid[iidx], arr[midx][1], arr[midx][0], lb, 
                                                         stninfo.loc[stnid[iidx]]["id"], stninfo.loc[stnid[iidx]]["t1"], stninfo.loc[stnid[iidx]]["t2"]))

                        monthly[iidx, yidx, midx] = arr[midx][1]
            else:
                logging.warning("{} doesn't exist!".format(inf))
    
    outdf = np.vstack([np.reshape(stnid, (1, -1)),
                       np.reshape(stninfo["lon"].values, (1, -1)),
                       np.reshape(stninfo["lat"].values, (1, -1)),
                       np.reshape(stninfo["elev"].values, (1, -1)),
                       np.reshape(monthly, (nstn, nyear * 12)).T])
    
    outdf = pd.DataFrame(outdf)
    
    idx = ["stnid", "lon", "lat", "elev"]
    idx.reverse()
    for idx_ in idx:
        dtimes.insert(0, idx_)
    outdf.index = dtimes
    outdf.columns = outdf.loc["stnid"]
    outdf.drop(index=["stnid"], inplace=True)
    
#     outdf.to_csv("{}/smarker.csv".format(outd))

    if stnkeep is not None:
       
        stnid = outdf.columns.tolist()
        # mnd.drop(index=["lon", "lat", "elev"], inplace=True)
        keepit = [str(i) for i in stnkeep]
        dropit = np.setdiff1d(stnid, keepit)
        
        for i in outdf.index.tolist()[3:]:
            outdf.loc[i][dropit] = -999
    
    
    outdf.to_csv(outf)

    
    return outdf

In [163]:
class textloader():
    def __init__(self, ind, outd="output", logd="log"):
        self.ind  = ind
        self.outd = outd
        
    @staticmethod
    def get_id(fname):
        try:
            bincode = codecs.open(fname, mode="r", encoding="big5")  # default encoding='utf-8', unicode transformation format
        except Exception as e :
#             logging.error("textloader-getid-11: ", e)
            logging.error(e)
            sys.exit(-1)

        stnlist_ = pd.read_csv(bincode, sep="\s+", header=0) 
        stnlist_.columns = ["#SN", "stnid", "lon", "lat", "elev", "t1", "t2", "id", "chname"]
#         return(stnlist_["stnid"].values)
        return(stnlist_["stnid"].values.tolist())
    
    @staticmethod
    def getGI(fname, index_col="#SN"):
        try:
            bincode = codecs.open(fname, mode="r", encoding="big5")  # default encoding='utf-8', unicode transformation format
        except Exception as e :
            logging.error(e)
            sys.exit(-1)

        stnlist_ = pd.read_csv(bincode, sep="\s+", header=0)     
        stnlist_.columns = ["#SN", "stnid", "lon", "lat", "elev", "t1", "t2", "id", "chname"]
        if index_col is not None:
            stnlist_.set_index(index_col, inplace=True)

        return(stnlist_)
    
    
    @staticmethod
    def cmtval(ind, outd, stnid, syr, eyr, hrfp="hr_rr", mnfp="mn_rr", showarn=True):
        '''
            calculating cmt val for precp (hourly to monthly)
            inf: old hr format, header is stn_id, yyyymmdd, 01, ..., 24
            hrfp: prefix of hourly data (input) 
            mnfp: prefix of monthly data (output)
        '''
        
        mday = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        
        hrfprefix = hrfp
        mnfprefix = mnfp 

        MNDA = np.ndarray((len(stnid), 12), dtype=np.float64)  # monthly data array
        NHR  = np.ndarray((len(stnid), 12), dtype=np.int64)    # number of hours in month

        MNDA.fill(0.0)
        NHR.fill(0)

        years = [syr + idx for idx in range(eyr - syr + 1)]

        miss_val = []
        for idx, id_ in enumerate(stnid):
            for YYYY in years:
                
                fname = "{0}_{1}_{2}.txt".format(hrfprefix, str(YYYY), id_)
                inf = os.path.join(ind, str(YYYY), fname)

                if not os.path.exists(inf):
                    if showarn:
#                         logging.warning("textloader-cmtval-50: {} doesn't exist!".format(inf))
                        logging.warning("{} doesn't exist!".format(inf))
                    continue

                df = pd.read_csv(inf, header=0, sep="\s+")
                df["date"] = pd.to_datetime(df["yyyymmdd"], format="%Y%m%d")
                df.set_index("date", inplace=True)
                df.drop(["stn_id", "yyyymmdd"], axis=1, inplace=True)

                fid = open("{0}/MON/{1}_{2}_{3}.txt".format(outd.strip(), mnfprefix, str(YYYY), id_), "w")

                for mm in range(12):
                    fid.write("{0:04d}{1:02d} ".format(YYYY, mm + 1))
                    t1 = "{0:04d}{1:02d}01".format(YYYY, mm + 1)
                    t2 = "{0:04d}{1:02d}{2:02d}".format(YYYY, mm + 1, mday[mm])
                    if df.index[-1].is_leap_year and mm + 1 == 2:  # for February in leap year
                        t2 = "{0:04d}{1:02d}{2:02d}".format(YYYY, mm + 1, mday[mm] + 1)

                    submat = df.loc[t1:t2].values
                    missing_idx = np.where(submat < 0)
                    submat_ = submat[np.where(submat >= 0)]
                    if submat_.size <= 0:
                        MNDA[idx, mm] += 0.0
                        NHR[idx, mm] += 0
                        fid.write("{0:>8.1f} {1:>8d}\n".format(-999, 0))
                    else:
                        MNDA[idx, mm] += np.nansum(submat_)
                        NHR[idx, mm] += 1
                        fid.write("{0:>8.1f} {1:>8d}\n".format(np.nansum(submat_), submat_.size))

                fid.close()

                nparr = df.values
                miss_val_ = np.unique(nparr[nparr < 0])
                for val_ in miss_val_:
                    if val_ not in miss_val:
                        miss_val.append(val_)

        # cmtval output
        if not os.path.exists(outd + "/CV"):
            os.makedirs(outd + "/CV")

        with open("{0}/CV/CV_M_RR.txt".format(outd.strip()), "w") as fid:
            fid.write("#stnId ")
            for mm in range(12):
                fid.write("      {0:>02d} ".format(mm + 1))
            fid.write("\n")
            for idx, id_ in enumerate(stnid): 
                fid.write("{0:6s} ".format(id_))
                for mm in range(12):
                    if NHR[idx, mm] > 0:
                        MNDA[idx, mm] = MNDA[idx, mm] / float(NHR[idx, mm])
                    else:
                        MNDA[idx, mm] = -999.0
                    fid.write("{0:>8.1f} ".format(MNDA[idx, mm]))
                fid.write("\n")

        with open("{0}/CV/NCV_M_RR.txt".format(outd.strip()), "w") as fid:
            fid.write("#stnId ")
            for mm in range(12):
                fid.write("      {0:>02d} ".format(mm + 1))
            fid.write("\n")
            for idx, id_ in enumerate(stnid): 
                fid.write("{0:6s} ".format(id_))
                for mm in range(12):
                    if NHR[idx, mm] > 0:
                        fid.write("{0:>8d} ".format(NHR[idx, mm]))
                    else:
                        fid.write("{0:>8d} ".format(-999))
                fid.write("\n")

        return [MNDA, NHR, miss_val]
    
    @staticmethod
    def _fdf(fname):
#         fname = "{0}/{1}/{2}_{1}_{3}.txt".format(ind, YYYY, hrfp, id_)
        if not os.path.exists(fname):
#             logging.warning("textloader-hrf-140: {} doesn't exist!".format(fname))
            logging.warning("{} doesn't exist!".format(fname))
            return None

        df = pd.read_csv(fname, header=0, sep="\s+")
        df["date"] = pd.to_datetime(df["yyyymmdd"], format="%Y%m%d")
        df.set_index("date", inplace=True)
        df.drop(["stn_id", "yyyymmdd"], axis=1, inplace=True)
        return df
    
    
    @staticmethod
    def hrf(ind, stnid, sdtime, edtime, hrfp="hr_rr", lb=0.0):

        '''
            load hourly data 
        '''
        
        syr = math.floor(sdtime / 10**6)
        eyr = math.floor(edtime / 10**6)

        YYYYmmddHH0000 = wg.YYYYmmddHH0000(syr, eyr, 1)
        YYYYmmddHH_ = [math.floor(YmdH / 10**4) for YmdH in YYYYmmddHH0000]

        sidx = YYYYmmddHH_.index(sdtime)
        eidx = YYYYmmddHH_.index(edtime)
        YYYYmmddHH = YYYYmmddHH_[sidx:(eidx + 1)]
#         logging.info("textloader-hrf-138: sdtime: {}, edtime: {}, nsample: {}".format(YYYYmmddHH[0], YYYYmmddHH[-1], len(YYYYmmddHH)))
        logging.info("sdtime: {}, edtime: {}, nsample: {}".format(YYYYmmddHH[0], YYYYmmddHH[-1], len(YYYYmmddHH)))

        
        # %%timeit -n 1

        nyr = eyr - syr + 1
        nstn = len(stnid)
        nsample = len(YYYYmmddHH)

        
        obs_ = np.ndarray(shape=(nstn, nyr, 366, 24))  # datetime to array index, don't use datetime to search index in YYYYmmddHH (not efficient)
        obs_.fill(-999)
        
        obs = np.ndarray(shape=(nstn, nsample))
        obs.fill(-999)

        years = [syr + i for i in range(nyr)]

        for ididx, id_ in enumerate(stnid):
#             dfs = []
            for yidx, YYYY in enumerate(years):
                fname = "{0}/{1}/{2}_{1}_{3}.txt".format(ind, YYYY, hrfp, id_)
                if not os.path.exists(fname):
                    logging.warning("textloader-hrf-156: {} doesn't exist!".format(fname))
                    logging.warning("{} doesn't exist!".format(fname))
                    continue

                df = pd.read_csv(fname, header=0, sep="\s+")
#                 df["date"] = pd.to_datetime(df["yyyymmdd"], format="%Y%m%d")
                df["date"] = df["yyyymmdd"]

                df.set_index("date", inplace=True)

                df.drop(["stn_id", "yyyymmdd"], axis=1, inplace=True)
#                 dfs.append(df)

#                 dtimes = df.index.values
                dtimes = df.index.tolist()

                for dtime in dtimes:
                  
                    Y_, m_, d_, H_ = wg.datetime_split(dtime, hour_system=1) 

                    for H_ in range(24):
                        ttuple = datetime.strptime("{0:04d}{1:02d}{2:02d}".format(Y_, m_, d_), "%Y%m%d").timetuple()
#                         logging.debug("hrf-209: {0}, {1:04d}-{2:02d}-{3:02d} {4:02d}:00:00".format(dtime, Y_, m_, d_, H_))
                        obs_[ididx, yidx, ttuple[7] - 1, H_] = df.loc[dtime][H_]
                    
            # reshape
            idx = 0
            dtidx = 0
            for yidx, YYYY in enumerate(years):
                yday = 365
                if calendar.isleap(YYYY):
                    yday += 1
#                 logging.debug("hrf-219: check yday={0}, dtidx={1}".format(yday, dtidx))
                logging.debug("check yday={0}, dtidx={1}".format(yday, dtidx))

                for didx in range(yday):
                    for hidx in range(24):
                        YmdH = YYYYmmddHH_[idx]
                        idx += 1
                        if YYYYmmddHH[0] <= YmdH and YmdH <= YYYYmmddHH[-1]:
                            obs[ididx, dtidx] = obs_[ididx, yidx, didx, hidx]
                            dtidx += 1

                             
#             if len(dfs) == 0:
#                 continue

#             df = pd.concat(dfs, axis=0)

#             dtidx = 0
#             for YmdH in YYYYmmddHH:
#                 Ymd = math.floor(YmdH / 100.0)
#                 HH = YmdH - Ymd * 100 - 1
#                 obs[ididx, dtidx] = df.loc[str(Ymd)][HH]
#                 dtidx += 1

        quantity = np.where(obs >= lb)[0].shape[0] / obs.size

#         logging.info("textloader-hrf-quantity: {}".format(quantity))
        logging.info("hrf-quantity = {}".format(quantity))

        return [obs, stnid, YYYYmmddHH, quantity]
    
    @staticmethod
    def mnf(inf, stnid, sdtime, edtime, vname="VNAME", outd=None, cvd=None, lb=0.0, nmn=10, ftype='dbout'):

        '''
            load monthly data "db_data_{vname}.txt and 
                (1) reformat it to "{outd}/MON/{YYYY}/MONTHLY_{vname}_{YYYY}_{id_}.txt" if {outd} is not None
                (2) calculate cmt val and output to {cvd} if {cvd} is not None
                
            ftype: 
                "dbout": output of get_data.py, all data in one 
                "smarker": indexes include lon, lat, elev, datetime, and columns are stations
        '''
        
        syr  = math.floor(sdtime / 10**2)
        eyr  = math.floor(edtime / 10**2)
        nyr  = eyr - syr + 1
        nstn = len(stnid)

        YYYYmm_ = wg.YYYYmm(syr, eyr)
        sidx = YYYYmm_.index(sdtime)
        eidx = YYYYmm_.index(edtime)
        YYYYmm = YYYYmm_[sidx:(eidx + 1)]
        nsample = len(YYYYmm)

        # ndarray declare
        cv = np.ndarray((nstn, nyr, 12))
        cv.fill(-999)

        obs = np.ndarray((nstn, nsample))
        obs.fill(-999)

        # load db data
        if ftype == "smarker":
            df = pd.read_csv(inf, header=0)
            df.columns = ["datetime"] + df.columns.tolist()[1:]
            df.set_index("datetime", inplace=True)
            gi = df.iloc[0:3]
            df.drop(index=["lon", "lat", "elev"], inplace=True)
            df.index = [int(i) for i in df.index.to_list()]
        else:
            df = pd.read_csv(inf, sep="\s+", header=0)
            df.set_index("datetime", inplace=True)

        dfdt = df.index.to_list()
        sidx = dfdt.index(sdtime)
        eidx = dfdt.index(edtime)

        for ididx, id_ in enumerate(stnid):
            if id_ in df.columns:
                obs[ididx, :] = df[id_].values[sidx:(eidx + 1)]
                for Ym in YYYYmm:
                    Y_ = math.floor(Ym / 100.0)
                    m_ = Ym - Y_ * 100
                    idx = dfdt.index(Ym)
                    cv[ididx, Y_ - syr, m_ - 1] = df.loc[Ym][id_]
        
        if outd is not None:
            for yridx in range(nyr):
                Y_ = syr + yridx
                
                if not os.path.exists("{}/MON/{}/{}".format(outd, vname, Y_)):
                    os.makedirs("{}/MON/{}/{}".format(outd, vname, Y_))
                
                for ididx, id_ in enumerate(stnid):
                    with open("{0}/MON/{1}/{2}/MONTHLY_{1}_{2}_{3}.txt".format(outd, vname, Y_, id_), "w") as fid:
                        for m_ in range(12):
                            fid.write("{0:04d}{1:02d}{2:9.1f}\n".format(Y_, m_ + 1, cv[ididx, yridx, m_]))

        if cvd is not None:
            cv[np.where(cv < lb)] = np.nan
            cmt_val = np.ndarray((nstn, 12))
            cmt_val.fill(-999)
            mncounter = np.zeros((nstn, 12), dtype=np.int32)
            for ididx in range(nstn):
                cmt_val[ididx, :] = np.nanmean(cv[ididx, :, :], axis=0)
                mncounter[ididx, :] = (~np.isnan(cv[ididx, :, :])).sum(axis=0)

            cmt_val[np.isnan(cmt_val)] = -999

            cmt_val[np.where(mncounter < nmn)] = -999
            
            if not os.path.exists(cvd):
                os.makedirs(cvd)

            cvfid  = open("{0}/CV_MN_{1}.txt".format(cvd, vname), "w")
            ncvfid = open("{0}/NCV_MN_{1}.txt".format(cvd, vname), "w")

            cvfid.write("#stnId")
            ncvfid.write("#stnId")
            if ftype == "smarker":
                cvfid.write("{0:>12s}".format("lon"))
                cvfid.write("{0:>12s}".format("lat"))
                cvfid.write("{0:>12s}".format("elev"))

            for mnidx in range(12):
                cvfid.write("{0:7s}{1:02d}".format(" ", mnidx + 1))
                ncvfid.write("{0:7s}{1:02d}".format(" ", mnidx + 1))
            cvfid.write("\n")
            ncvfid.write("\n")

            for ididx in range(nstn):
                cvfid.write(stnid[ididx])
                if ftype == "smarker":
                    cvfid.write("{0:12.4f}".format(gi.iloc[0][stnid[ididx]]))
                    cvfid.write("{0:12.4f}".format(gi.iloc[1][stnid[ididx]]))
                    cvfid.write("{0:12.4f}".format(gi.iloc[2][stnid[ididx]]))
                ncvfid.write(stnid[ididx])
                
                for mnidx in range(12):
                    cvfid.write("{0:9.1f}".format(cmt_val[ididx, mnidx]))
                    ncvfid.write("{0:9d}".format(mncounter[ididx, mnidx]))
                cvfid.write("\n")
                ncvfid.write("\n")

            cvfid.close()
            ncvfid.close()

        quantity = np.where(obs >= lb)[0].shape[0] / obs.size

        logging.info("mnf-quantity = {}".format(quantity))

        return [obs, stnid, YYYYmm, quantity]

    @staticmethod
    def floader(fname, nvar, qkey=None):
        df = pd.read_csv(fname, sep="\s+", header=None)
        if nvar == 3:
            df.columns = ["id", "lon", "lat", "elev", "obs", "est", "thmval"]
        else:
            df.columns = ["id", "lon", "lat", "elev", "obs", "est"]
        df.set_index("id", inplace=True)
        
        return [df, qkey]
        
    @staticmethod
    def ossef(ind, stnid, nvar, syr, eyr, fprefix="OSSE_", **kwargs):

        '''
            load output of osse for hourly, daily and monthly data
            nvar: 3 for obs, est and thmval and 2 for obs and est
        '''

        nyr = eyr - syr + 1
        Ymd = wg.YYYYmmdd(syr, eyr)
        ndt = len(Ymd)
        nstn = len(stnid)

        HH = [1 + i for i in range(24)]
        YYYY = [syr + i for i in range(nyr)]

        osseH = np.ndarray((24, 366, nyr, nstn, nvar))
        osseH.fill(-999)
        logging.info("load osse for hourly data")
        jobs = []
        YmdHs = []
        for dt_idx, Ymd_ in enumerate(Ymd):
#             dtuple = datetime.strptime(str(Ymd_), "%Y%m%d").timetuple()
#             Y_ = dtuple[0] 
#             M_ = dtuple[1]
#             wday = dtuple[6]
            wg.display_progress(dt_idx, ndt, 2)
            for H_ in HH:
#                 df = pd.read_csv("{0}/{1}{2}{3:02d}.txt".format(ind, fprefix, Ymd_, H_), sep="\s+", header=None)
#                 if nvar == 3:
#                     df.columns = ["id", "lon", "lat", "elev", "obs", "est", "thmval"]
#                 else:
#                     df.columns = ["id", "lon", "lat", "elev", "obs", "est"]
#                 df.set_index("id", inplace=True)
#                 for id_idx, id_ in enumerate(stnid):
#                     osseH[H_ - 1, wday - 1, Y_ - syr, id_idx, 0] = df.loc[id_]["obs"]
#                     osseH[H_ - 1, wday - 1, Y_ - syr, id_idx, 1] = df.loc[id_]["est"]
#                     if nvar == 3:
#                         osseH[H_ - 1, wday - 1, Y_ - syr, id_idx, 2] = df.loc[id_]["thmval"]

                YmdHs.append(Ymd_ * 100 + H_)
                jobs.append([textloader.floader, ["{0}/{1}{2}{3:02d}.txt".format(ind, fprefix, Ymd_, H_), nvar, YmdHs[-1]]])
                
        ret = runFunctionsInParallel(jobs, names=[str(i) for i in YmdHs],
                                     offsetsSeconds=0.2, parallel=True, allowJobFailure=True)
        
        for jidx in range(len(YmdHs)):
            df = ret[1][jidx][0]
            qkey = ret[1][jidx][1]  # datetime, YYYYmmddHH
            logging.info("queue for {}".format(qkey))
            Ymd_ = str(qkey)[0:-2]
            H_ = int(str(qkey)[-2:])
            dtuple = datetime.strptime(Ymd_, "%Y%m%d").timetuple()
            Y_ = dtuple[0] 
            M_ = dtuple[1]
#             H_ = dtuple[3]
            wday = dtuple[7]            
            for id_idx, id_ in enumerate(stnid):
                osseH[H_ - 1, wday - 1, Y_ - syr, id_idx, 0] = df.loc[id_]["obs"]
                osseH[H_ - 1, wday - 1, Y_ - syr, id_idx, 1] = df.loc[id_]["est"]
                if nvar == 3:
                    osseH[H_ - 1, wday - 1, Y_ - syr, id_idx, 2] = df.loc[id_]["thmval"]

        gc.collect()        

                    
        osseD = np.ndarray((366, nyr, nstn, nvar))
        osseD.fill(-999)
        logging.info("load osse for daily data")
        Ymds = []
        for dt_idx, Ymd_ in enumerate(Ymd):
            Ymds.append(Ymd_)
            dtuple = datetime.strptime(str(Ymd_), "%Y%m%d").timetuple()
            Y_ = dtuple[0] 
            M_ = dtuple[1]
            wday = dtuple[7]
            wg.display_progress(dt_idx, ndt, 2)
            df = pd.read_csv("{0}/{1}{2}.txt".format(ind, fprefix, Ymd_), sep="\s+", header=None)
            if nvar == 3:
                df.columns = ["id", "lon", "lat", "elev", "obs", "est", "thmval"]
            else:
                df.columns = ["id", "lon", "lat", "elev", "obs", "est"]
            df.set_index("id", inplace=True)
            for id_idx, id_ in enumerate(stnid):
                osseD[wday - 1, Y_ - syr, id_idx, 0] = df.loc[id_]["obs"]
                osseD[wday - 1, Y_ - syr, id_idx, 1] = df.loc[id_]["est"]
                if nvar == 3:
                    osseD[wday - 1, Y_ - syr, id_idx, 2] = df.loc[id_]["thmval"]

        gc.collect()        

                    
        osseM = np.ndarray((12, nyr, nstn, nvar))
        osseM.fill(-999)
        lcounter = 0  # for progress display
        logging.info("load osse for monthly data")
        Yms = []
        for yr_idx, Y_ in enumerate(YYYY):
            for m_ in range(12):
                Yms.append(Y_ * 100 + m_ + 1)
                wg.display_progress(lcounter, nyr * 12, 2)
                lcounter += 1
                df = pd.read_csv("{0}/{1}{2}{3:02d}.txt".format(ind, fprefix, Y_, m_ + 1), sep="\s+", header=None)
                if nvar == 3:
                    df.columns = ["id", "lon", "lat", "elev", "obs", "est", "thmval"]
                else:
                    df.columns = ["id", "lon", "lat", "elev", "obs", "est"]
                df.set_index("id", inplace=True)
                for id_idx, id_ in enumerate(stnid):
                    osseM[m_, yr_idx, id_idx, 0] = df.loc[id_]["obs"]
                    osseM[m_, yr_idx, id_idx, 1] = df.loc[id_]["est"]
                    if nvar == 3:
                        osseM[m_, yr_idx, id_idx, 2] = df.loc[id_]["thmval"]

        gc.collect()        

                        
        return [osseH, osseD, osseM, YmdHs, Ymds, Yms]
    

# staticmethod test

In [183]:
if __name__ == "__main__":
    
#     logging.getLogger().setLevel(logging.DEBUG)

    targetf = "cmtval"
#     targetf = "ossef"
    targetf = "dec2anls"
    
    if targetf == "hr2mn":
        ind     = "/NAS-129/users1/T1/DATA/YY/ORG/HR1/RR"
        outd    = "/NAS-129/users1/T1/DATA/YY/ORG"
        stninfo = "/home/yuzhe/CODE/ProgramT1/GRDTools/SRC/RES/GI/RR_analysis_grid_stationlist.txt"
        fprefix = "hr_rr"
        syr = 1998
        eyr = 2018
        # stnid = textloader.get_id(stninfo)
        # MNDA, NHR, miss_val = textloader.cmtval(ind, outd, stnid, syr, eyr, hrfp=fprefix, showarn=False)
    elif targetf == "cmtval":
        # load monthly data (format: all in one) and calculate climate values
        stninfo = "/home/yuzhe/CODE/ProgramT1/GRDTools/SRC/RES/GI/RR_analysis_grid_stationlist.txt"
    #     inf = "/NAS-129/users1/T1/DATA/YY/ORG/MON/db_data_Precp.txt"
    #     inf = "/NAS-129/users1/T1/DATA/YY/ORG/MON/Precp_1981_2019.txt"
    #     outd = '/NAS-129/users1/T1/DATA/YY/ORG'

        stnid = textloader.get_id(stninfo)

        # cvd=None, get monthly data (1981 ~ 1997, 1998 ~ 2019 will be replace by using preprocessing data)
        sdtime = 198101
        edtime = 201912
        inf = "/NAS-129/users1/T1/DATA/YY/CXN/MON/Precp_1981_2019.txt"  # use ORG with CXN procedure
        outd = '/NAS-129/users1/T1/DATA/YY/CXN' 
        obs, stnid, YYYYmm, quantity = textloader.mnf(inf, stnid, sdtime=sdtime, edtime=edtime, vname="RR", outd=outd, cvd=None, lb=0.0)

        # get cmt val and replace monthly data (1998 ~ 2019)
        sdtime = 199801
        edtime = 201812
        cvd = '/NAS-129/users1/T1/DATA/YY/ORG/CV'  # 1998 ~ 2018
        inf = "/NAS-129/users1/T1/DATA/Preprocess/MON/RR/smarker.csv"
        outd = '/NAS-129/users1/T1/DATA/YY/CXN'
        obs, stnid, YYYYmm, quantity = textloader.mnf(inf, stnid, sdtime=sdtime, edtime=edtime, vname="RR", outd=outd, cvd=cvd, lb=0.0, ftype="smarker")
                
    elif targetf == "hrf":
        ind = "/NAS-129/users1/T1/DATA/YY/ORG/HR1/RR"
        sdtime = 1998010101
        edtime = 1999013124
        hrfp = "hr_rr"
        stnid = textloader.get_id(stninfo)
    #     obs, stnid, YYYYmmddHH, quantity = textloader.hrf(ind, stnid, sdtime=sdtime, edtime=edtime, hrfp=hrfp)
    elif targetf == "ossef":
        wg.get_pentad(20120302, 123)    
        ind = "/NAS-129/users1/T1/DATA/GRD/OSSE/UNC/OSSE/PP"
        gid = "/NAS-129/users1/T1/DATA/YY/GI/PP_analysis_grid_stationlist.txt"
        syr = 1998
        eyr = 2017
        nvar = 3
        stnid = textloader.get_id(gid)
        osseH, osseD, osseM = textloader.ossef(ind, stnid, nvar, 2011, 2013)
    
    elif targetf == "idSelet":
        stnf = "/home/yuzhe/CODE/ProgramT1/GRDTools/SRC/RES/GI/RR_analysis_grid_stationlist.txt"
        fname = "/NAS-129/users1/T1/DATA/YY/CXN/MON/RR/smarker.csv" 
        idSelect(stnf, fname, key="198101", val=0, outf="sinfo1981.txt")
        
    elif targetf == "dec2anls":
        stnf = "/NAS-129/users1/T1/DATA/YY/GI/RH_decode_grid_stationlist_2012_2019.txt"
        sinfo = idSelect(stnf, outf="/NAS-129/users1/T1/DATA/YY/GI/RH_analysis_grid_stationlist_2012_2019.txt")



In [173]:
for i in sinfo.iloc[0].tolist():
    print(i)

1
466880
121.4421
24.9976
9.7
2002010101
2019123199
1
ChineseName


In [176]:
sinfo
# 0001 466900 121.4489 25.1649   19.0000 1911010101 2018051099 1 淡水

with open("test.txt", "w") as fid:
    fid.write("{0:4s} {1:6s} {2:8s} {3:7s} {4:9s} {5:10s} {6:10s} {7:2s} {8}\n".format("#SN", "stnid", "lon", "lat", "elev", "t1", "t2", "id", "chname"))
    for idx in range(sinfo.shape[0]):
        rc = sinfo.iloc[idx].tolist()
        fid.write("{0:04d} {1:6s} {2:8.4f} {3:7.4f} {4:9.4f} {5:10d} {6:10d} {7:2d} {8}\n".format(rc[0], rc[1], rc[2], rc[3], rc[4], rc[5], rc[6], rc[7], rc[8]))
            

# build smarker for gridding map (combine all monthly data into one file)

In [157]:
if __name__ == "__main__":
    
    stnf = "/home/yuzhe/CODE/ProgramT1/GRDTools/SRC/RES/GI/RR_analysis_grid_stationlist.txt"
    vname = "RR"

#     stnkeep = "/home/yuzhe/CODE/ProgramT1/GRDTools/SRC/RES/GI
    stnkeepf = "/NAS-129/users1/T1/DATA/YY/GI/sinfo1981.txt"
    stnkeep = pd.read_csv(stnkeepf, header=0)
    stnkeep = stnkeep["stnid"].tolist()
    
    # condition
    smarkerf = "GConsistency"
#     smarkerf = "GTraceback"

    if smarkerf == "GConsistency":
        ind = "/NAS-129/users1/T1/DATA/Preprocess/MON"
        tperiod = [1998, 2019]

    elif smarkerf == "GTraceback":
#         ind = "/NAS-129/users1/T1/DATA/YY/ORG/MON"
        ind = "/NAS-129/users1/T1/DATA/YY/CXN/MON"
        tperiod = [1981, 1997]
    
    outf = "{}/{}/smarker.csv".format(ind, vname)
    mnd = mnf2onef(ind, outf, stnf, tperiod, vname)
    
#     outf = "{}/{}/smarker.csv".format(ind, vname)
#     mnd = mnf2onef(ind, outf, stnf, tperiod, vname, stnkeep=stnkeep)
#     mnd